## Домашнее задание

Основаная задача - **построить хорошую тематическую модель с интерпретируемыми топиками с помощью LDA в gensim и NMF в sklearn**.


1) сделайте нормализацию (если pymorphy2 работает долго используйте mystem или попробуйте установить быструю версию - `pip install pymorphy2[fast]`, можно использовать какой-то другой токенизатор); 

2) добавьте нграммы (в тетрадке есть закомменченая ячейка с Phrases,  можно также попробовать другие способы построить нграммы); 

3) сделайте хороший словарь (отфильтруйте слишком частотные и редкие слова, попробуйте удалить стоп-слова); 

4) постройте несколько LDA моделей (переберите количество тем, можете поменять eta, alpha, passes), если получаются плохие темы, поработайте дополнительно над предобработкой и словарем; 

5) для самой хорошей модели в отдельной ячейке напечатайте 3 хороших (на ваш вкус) темы;

6) между словарем и обучением модели добавьте tfidf (`gensim.models.TfidfModel(corpus, id2word=dictionary); corpus = tfidf[corpus]`);

7) повторите пункт 4 на преобразованном корпусе;

8) в отдельной ячейке опишите как изменилась модель (приведите несколько тем, которые стали лучше или хуже, или которых раньше вообще не было; можно привести значения перплексии и когерентности для обеих моделей)

9) проделайте такие же действия для NMF (образец в конце тетрадки), для построения словаря воспользуйтесь возможностями Count или Tfidf Vectorizer (попробуйте другие значение max_features, min_df, max_df, сделайте нграмы через ngram_range, если хватает памяти), попробуйте такие же количества тем

10) в отдельной ячейки напечатайте таблицу с темами лучшей NMF модели, сравните их с теми, что получились в LDA.

Сохраните тетрадку с экспериментами и положите её на гитхаб, ссылку на неё укажите в форме.

**Оцениваться будут главным образом пункты 5, 8 и 10. (2, 3, 2 баллов соответственно). Чтобы заработать остальные 3 балла, нужно хотя бы немного изменить мой код на промежуточных этапах (добавить что-то, указать другие параметры и т.д). **

In [1]:
import gensim
import json
import re
import pickle
import pyLDAvis.gensim
import string
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from itertools import chain
from pymystem3 import Mystem
from collections import Counter

import warnings
warnings.filterwarnings("ignore")

mystem = Mystem()

## Данные

In [2]:
stops = set(stopwords.words('russian')) | {'gt',}

def remove_tags(text):
    return re.sub(r'<[^>]+>', '', text)

def normalize(words):
    norm_words = [mystem.lemmatize(word)[0] for word in words if len(word) > 2 and word not in stops]
    return norm_words

def tokenize(text):
    words = [word.strip(string.punctuation) for word in text.split()]
    words = [word for word in words if word]
    
    return words

In [3]:
%%time
texts = open('habr_texts.txt').read().splitlines()
texts = [normalize(tokenize(remove_tags(text.lower()))) for text in texts]

CPU times: user 3min 35s, sys: 58.1 s, total: 4min 34s
Wall time: 8min 44s


In [4]:
with open('texts.json', 'w', encoding='utf-8') as fw:
    json.dump(texts, fw)

In [9]:
# для нграммов
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.4) # threshold можно подбирать
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts_4 = p[texts]

In [8]:
ngrammed_2 = set(chain.from_iterable(ngrammed_texts_2))

In [10]:
ngrammed_4 = set(chain.from_iterable(ngrammed_texts_4))

In [6]:
ngrammed_7 = set(chain.from_iterable(ngrammed_texts_7))

Сравним тексты, полученными со значением ```threshold```, равным 0.2, 0.4 и 0.7:

In [37]:
# пример первых 20 нграмм, которые есть в ngrammed_4 и нет в ngrammed_2
list(ngrammed_4.difference(ngrammed_2))[:20]

['целый_матрешка',
 'packed_double',
 'недосмотр_вроде',
 'возвращаться_ec2',
 'ctx_wndclassw',
 'dgst_sha256',
 'среда_gomaxprocs',
 'автономность_транка',
 'broadridge_планировать',
 'никод',
 'bsim19_кнопка',
 'a5d6a7_mdl',
 'мвт_ассортимент',
 'деление_нацело',
 'радиокнопка_взаимодействовать',
 'замороженый_деньги',
 'генерить',
 'gettransparent_write',
 'дурной_чувствовать',
 'познание_трехмерный']

В текстах с ```threshold = 0.4``` есть хорошие коллокации, например, *'вестминстерский_дворец'*, *'full\_installation'*, *'африканский\_кампания'*, *'похлопать\_плечо'*, которых нет в текстах с ```threshold = 0.2```.

In [36]:
# пример первых 20 нграмм, которые есть в ngrammed_7 и нет в ngrammed_4
list(ngrammed_7.difference(ngrammed_4))[:20]

['securityz',
 '1200t',
 'cameradevice',
 'ymin',
 'doty',
 'ipb',
 'угнетаться',
 'metropolitana',
 'objectsatendinterval',
 'ещёsudo_binwalk',
 'слеплять',
 'myspace',
 'l40x2',
 'нескончаемый',
 'безвыходный',
 'пирс',
 'новостиupd',
 'токарная',
 'выкрикивать',
 'противность']

Сравнивая тексты с ```threshold = 0.4``` и ```threshold = 0.7```, уже сложнее найти хорошие коллокации при ```threshold = 0.7```, которых нет при ```threshold = 0.4```, но некоторые находятся, например: *'зашевилиться_волос'* и *'остросюжетный_детектив'*.

### Тематическое моделирование в gensim

In [23]:
def get_lda_model(texts, num_topics=100, passes=5, eta='auto', id2word=None, not_return_id2word=False, 
                     return_corpus=False):
    if not id2word:
        id2word = gensim.corpora.Dictionary(texts)
        id2word.filter_extremes(no_above=0.3)
        id2word.compactify()
    
        corpus = [id2word.doc2bow(text) for text in texts]
    else:
        corpus = texts
    
    lda = gensim.models.LdaMulticore(corpus, num_topics, id2word=id2word, passes=passes, eta=eta,
                                     iterations=10)
    
    print('Perplexity: ', lda.log_perplexity(corpus[:2000], total_docs=100))
    
    if return_corpus:
        return lda, id2word, corpus
    elif not_return_id2word:
        return lda
    else:
        return lda, id2word

In [24]:
def get_coherence(lda, dictionary, texts, n_col):
        
    topics = []
    word_dict = {}
    for topic_id, topic in lda.show_topics(num_topics=100, formatted=False):
        topic = [word for word, _ in topic]
        topics.append(topic)
        word_dict['Topic # ' + '{:02d}'.format(topic_id)] = topic
    
    coherence_model_lda = gensim.models.CoherenceModel(topics=topics, 
                                                       texts=texts, 
                                                       dictionary=dictionary, coherence='c_v')
    coherence = coherence_model_lda.get_coherence()
    print('Coherence: ', coherence)
    
    return coherence, pd.DataFrame(word_dict).iloc[:, 0:n_col]

Сравним результаты с разным ```threshold``` у нграмм:

In [50]:
lda_model_4, dictionary_4 = get_lda_model(ngrammed_texts_4)
c, topics = get_coherence(lda_model_4, dictionary_4, ngrammed_texts_4, 20)
topics

Coherence:  0.46843378662142304


,Topic # 00,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,Topic # 11,Topic # 12,Topic # 13,Topic # 14,Topic # 15,Topic # 16,Topic # 17,Topic # 18,Topic # 19
0,значение,сервер,java,оборудование,устройство,метод,return,up,import_java,сигнал,значение,монета,продукт,доклад,сборщик,lt,объект,сервис,игра,игра
1,ошибка,сеть,язык,устройство,сеть,список,объект,2016,system,усилитель,пиксел,источник,сотрудник,тема,память,int,элемент,сервер,приставка,какой-то
2,список,интернет,ядро,модуль,доступ,поддержка,изображение,какой-то,void_main,частота,модель,view,цель,выступление,программа,for_int,домен,программа,nintendo,клиент
3,файл,клиент,память,контроллер,номер,пакет,epoch,книга,new,бумага,временный_ряд,сигнал,сайт,игрок,алгоритм,значение,файл,файл,устройство,pebble
4,поле,сервис,ребенок,программа,сервер,qml,запрос,заниматься,public_void,искажение,точка,сайт,клиент,письмо,ssd,if,вызов,сайт,робот,точка
5,запрос,доступ,программа,дата,значение,модуль,боль,12,public_static,астероид,ита,изображение,идея,враг,размер,long_long,программа,клиент,доллар,pc
6,элемент,google,телефон,компьютер,верификация,qt,класс,11,покупка,звук,объект,устройство,что-то,конференция,метод,return,сайт,сообщение,сеть,еще
7,php,трафик,openmp,управление,бизнес,if,type,час,if,обратный_связь,базовый,кит,бизнес,игра,пауза,элемент,страница,доступ,технология,набор
8,print,пакет,язык_программирование,питание,спускаться,запускать,“,проходить,абонент,мощность,20,технология,программист,рассылка,сборка_мусор,foo,ошибка,сеть,deepmind,ошибка
9,lt,связь,компилятор,лазер,4,lt,if,путь,in,гармоника,область,метод,опыт,подписчик,lt,объект,opencv,интернет,агент,мир


In [51]:
lda_model_7, dictionary_7 = get_lda_model(ngrammed_texts_7)
c, topics = get_coherence(lda_model_7, dictionary_7, ngrammed_texts_7, 20)
topics

Coherence:  0.5000414555311885


,Topic # 00,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,Topic # 11,Topic # 12,Topic # 13,Topic # 14,Topic # 15,Topic # 16,Topic # 17,Topic # 18,Topic # 19
0,значение,salt,компонент,сигнал,fortran,lt,машина,файл,ms,python,файл,глаз,файл,программа,value,сайт,модель,this,устройство,elapsed
1,товар,миньон,сервер,датчик,позиция,gulp,диск,агрегат,sec,apple,docker,операция,ошибка,клиент,lt,устройство,рынок,public,камера,sleep
2,пиксел,мастер,react,квантовый,makerbot,import,инструмент,устройство,5,операция,сервер,swift,программа,сайт,ключ,технология,устройство,return,смартфон,lt
3,font,ключ,this,значение,память,файл,станок,значение,sum,устройство,http,роговица,папка,язык,файл,бизнес,ассемблер,private,модель,test
4,100,rhel,return,транзистор,light,vendor,протез,sr,4,рынок,etc,scrum,значение,сеть,hello,интернет,стекло,new,телефон,std
5,20,файл,элемент,ток,еще,0.0,сервер,метод,адрес,программа,контейнер,лазер,библиотека,торрент,сайт,сеть,pocketbook,function,сайт,time
6,mb,red_hat,lt,канал,int,сервер,мотор,сеть,2.1,собственный,nginx,“,класс,интерфейс,pin,курс,скорость,void,цена,starting
7,размер,master,список,частота,the,ключ,измерение,кардиостимулятор,2.2,она,настройка,пациент,ядро,интернет,устройство,клиент,связь,объект,экран,значение
8,скидка,сервер,vertica,поле,простыня,for,размер,jmeter,подсеть,мозг,запускать,линза,int,какой-то,сертификат,обработка,технология,string,интернет,timer
9,1,запускать,if,точка,игра,запускать,текстура,метка,сеть,антимонопольный,https,хирург,тест,машина,сервер,google,сеть,if,безопасность,вывод


C ```threshold = 0.7``` значение ```coherence``` получилось выше всего. Попробуем дальше использовать именно этот ```threshold```.

Изменим предобработку: исключим из текстов цифры и двубуквенные слова.

In [2]:
stops = set(stopwords.words('russian'))
punct = string.punctuation + '»«–…”“' + string.digits

def remove_tags(text):
    return re.sub(r'<[^>]+>', '', text)

def normalize(words):
    norm_words = [mystem.lemmatize(word)[0] for word in words if len(word) > 2]
    norm_words = [word for word in norm_words if word not in stops]
    return norm_words

def tokenize(text):
    words = [word.strip(punct) for word in text.split()]
    words = [word for word in words if word]
    
    return words

In [3]:
texts_first = open('habr_texts.txt').read().splitlines()
texts = [normalize(tokenize(remove_tags(text.lower()))) for text in texts_first]

In [4]:
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.7)
p = gensim.models.phrases.Phraser(ph)
new_ngrammed_texts = p[texts]

In [5]:
with open('ngrammed_texts.pkl', 'wb') as fw:
    pickle.dump(new_ngrammed_texts, fw)

In [2]:
with open('ngrammed_texts.pkl', 'rb') as f:
    new_ngrammed_texts = pickle.load(f)

In [5]:
new_lda_model, new_dictionary = get_lda_model(new_ngrammed_texts)
c, topics = get_coherence(new_lda_model, new_dictionary, new_ngrammed_texts, 20)
topics

Coherence:  0.47911614481876064


,Topic # 00,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,Topic # 11,Topic # 12,Topic # 13,Topic # 14,Topic # 15,Topic # 16,Topic # 17,Topic # 18,Topic # 19
0,сообщение,значение,user,устройство,устройство,игра,запрос,файл,игра,сайт,спутник,ключ,лампа,память,фигура,return,письмо,javascript,доклад,запись
1,игра,поле,admin,компьютер,сеть,цель,сервер,null,игрок,сервер,ракета,файл,исследование,значение,игра,string,галактика,css,конференция,файл
2,какой-то,канал,conn,процессор,сетевой,visa,транзакция,сервис,игровой,хостинг,объект,pebble,игра,ядро,блокчейн,bool,адрес,точка,microsoft,int
3,мой,сигнал,test,ноутбук,сервер,тест,сайт,уведомление,puppet,скидка,земля,сервер,популяция,процессор,поле,result,отправитель,язык,сервис,таблица
4,список,сеть,домен,windows,доступ,исследование,база,can,играть,клиент,орбита,сайт,испытуемый,поток,форма,int,файл,страница,vagrant,элемент
5,читать,устройство,role,рынок,интернет,устройство,кластер,запрос,персонаж,домен,космический,подпись,еще,цикл,биткойн,else,заголовок,веб,devops,for
6,сайт,переменный,тест,доллар,услуга,язык,хранение,not,история,the,пространство,открытый,какой-то,инструкция,двигатель,класс,значение,координата,тестирование,блок
7,приходить,усилитель,сайт,сегодня,безопасность,сеть,сервис,сервер,salt,акция,язык,веб,мозг,программа,еще,byte,домен,ребро,тема,база
8,канал,адрес,файл,час,лекция,программа,восстановление,управление,сервер,vps,планета,злоумышленник,физический,компилятор,ошибка,void,сервер,луч,база,операция
9,заниматься,частота,зона,microsoft,очередь,неуловимый,ключ,set,миньон,реклама,php,безопасность,современный,int,метод,const,require,sass,data,log


Значение ```coherence``` немного уменьшилось, но зато теперь нет непонятных слов.

Посмотрим, как будет влиять число тем на ```coherence```:

In [10]:
for i in list(range(10, 100, 20)) + list(range(100, 400, 100)):
    model, dictionary = get_lda_model(new_ngrammed_texts, num_topics=i)
    print('n_topics: ', i)
    c, _ = get_coherence(model, dictionary, new_ngrammed_texts, 20)

n_topics:  10
Coherence:  0.4010143289893584
n_topics:  30
Coherence:  0.47501931671571496
n_topics:  50
Coherence:  0.47559195912270347
n_topics:  70
Coherence:  0.480704778119817
n_topics:  90
Coherence:  0.47961027623006985
n_topics:  100
Coherence:  0.4644657910435386
n_topics:  200
Coherence:  0.4710407387468789
n_topics:  300
Coherence:  0.44355558574404313


Лучшее значение ```coherence``` получается при ```n_topics = 70```. Далее подберем параметр ```eta```:

In [11]:
for i in ['auto', 0.0001, 0.001, 0.01, 1]:
    model, dictionary = get_lda_model(new_ngrammed_texts, num_topics=70, eta=i)
    print('eta: ', i)
    c, _ = get_coherence(model, dictionary, new_ngrammed_texts, 20)

eta:  auto
Coherence:  0.44883517373953136
eta:  0.0001
Coherence:  0.49950012071975536
eta:  0.001
Coherence:  0.4780575818101345
eta:  0.01
Coherence:  0.47699349066894964
eta:  1
Coherence:  0.43396530103294


В этом случае лучшее значение ```coherence``` получается при ```eta = 0.0001```. Далее подберем параметр ```passes```:

In [13]:
for i in range(5, 20, 5):
    model, dictionary = get_lda_model(new_ngrammed_texts, num_topics=70, eta=0.0001, passes=i)
    print('passes: ', i)
    c, _ = get_coherence(model, dictionary, new_ngrammed_texts, 20)

passes:  5
Coherence:  0.48960886053795505
passes:  10
Coherence:  0.5468845248928476
passes:  15
Coherence:  0.5798874449347049


In [31]:
new_lda_model2, new_dictionary2, new_corpus2 = get_lda_model(new_ngrammed_texts, num_topics=70, eta=0.0001,
                                                                passes=15, return_corpus=True)
c, topics = get_coherence(new_lda_model2, new_dictionary2, new_ngrammed_texts, 20)
topics

Perplexity:  -90.86654043193215
Coherence:  0.5428529646433754


,Topic # 00,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,Topic # 11,Topic # 12,Topic # 13,Topic # 14,Topic # 15,Topic # 16,Topic # 17,Topic # 18,Topic # 19
0,проектирование,глаз,товар,устройство,память,файл,the,язык,анимация,php,docker,безопасность,d,игра,доклад,сервер,библиотека,адрес,значение,транзакция
1,сайт,операция,магазин,камера,int,http,сертификат,регистр,var,база,amp,сеть,печать,игрок,конференция,сервис,ассемблер,маска,алгоритм,tesla
2,этап,нейрон,amazon,смартфон,процессор,сервер,pvs,контекст,токен,linux,контейнер,интернет,материал,игровой,тема,запись,загрузчик,сеть,элемент,блокировка
3,прототип,сигнал,покупатель,экран,цикл,запускать,анализатор,scala,int,процессор,nginx,атака,принтер,играть,выступление,хостинг,лекция,подсеть,int,pc
4,интерфейс,квантовый,доставка,модель,программа,скрипт,ошибка,значение,ключ,архитектура,client,доступ,производство,фильм,слайд,запрос,opencv,бит,список,субд
5,страница,лазер,скидка,телефон,инструкция,настройка,ssl,предметный_область,элемент,сервер,lt,защита,деталь,персонаж,презентация,объем,компилятор,враг,массив,запрос
6,требование,роговица,заказ,ноутбук,компилятор,устанавливать,linux,ограниченный,for,производительность,if,устройство,модель,виртуальный_реальность,язык,управление,cmake,зона,блок,bitcoin
7,дизайнер,зрение,фотография,компьютер,значение,установка,root,сущность,eax,поддержка,run,связь,рабочий,история,участник,резервный_копия,торрент,блок,точка,postgresql
8,макет,метод,покупка,цена,sum,сайт,проверка,модель,function,программа,сервер,инфраструктура,станок,экран,net,dns,программа,игрок,for,ошибка
9,спринт,линза,продажа,производитель,for,https,предупреждение,агрегат,return,язык,char,оператор,изделие,движок,java,заявка,язык,укрытие,ячейка,инструмент


In [75]:
topics[['Topic # 02', 'Topic # 13', 'Topic # 15']]

,Topic # 02,Topic # 13,Topic # 15
0,энергия,пациент,диск
1,вселенная,клетка,запись
2,свет,глаз,чтение
3,объект,ученый,ssd
4,ученый,операция,накопитель
5,скорость,врач,участник
6,квантовый,лечение,набор
7,пространство,болезнь,configuration
8,частица,организм,оперативный_память
9,волна,заболевание,коммит


In [28]:
tfidf = gensim.models.TfidfModel(new_corpus2, id2word=new_dictionary2)
corpus = tfidf[new_corpus2]

In [78]:
tfidf_lda_model = get_lda_model(corpus, id2word=new_dictionary2,
                                                  num_topics=70, eta=0.0001,
                                                  passes=15, not_return_id2word=True)
c, topics = get_coherence(tfidf_lda_model, new_dictionary2, new_ngrammed_texts, 20)
topics

Perplexity:  -585.6361788529076
Coherence:  0.4754481739540101


,Topic # 00,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,Topic # 11,Topic # 12,Topic # 13,Topic # 14,Topic # 15,Topic # 16,Topic # 17,Topic # 18,Topic # 19
0,лед,void,electron,print,капча,регистратор,маяк,asterisk,nvme,vsphere,hyperx,измерение,вакансия,банк,sql_server,ураган,университет_итмо,nvarchar,вселенная,таймер
1,водород,public,печень,root,activation,blackvue,ble,виза,трансляция,random,pla,прибор,отклик,услуга,openmp,ртуть,params,srv,космический,gpio_gpio
2,tensorflow,ionic,хромосома,cmd,тьюринг,подложка,оконечный,attributes,vps,laravel,ddr,кандидат,эпидемия,платеж,потепление,кубит,итератор,протон,спутник,king
3,благотворительный,кремний,пластинка,echo,oem,блокировщик,gulp,алмаз,fjs,else,ротация,сердце,небоскреб,затрата,тень,банкомат,pg,elixir,планета,gpio
4,кратер,рекурсия,срок_годность,include,болезнь_альцгеймер,train,лига,белорусский,гиперконвергентный,gen,phpixie,сетка,руб,azure,css,экзамен,null,mix,орбита,кардиостимулятор
5,seagate,sha256,iptables,usr,дыра,opera,шорткат,фотонный,телеком,иннополис,kingston,собеседование,интеграл,узел,многоугольник,пленка,postgresql,стакан,галактика,свч
6,диплом,печкин,монета,file,hotspot,оксид,фишер,foundry,selenium,middleware,фольга,гравитация,откликаться,продвижение,гражданство,преподавать,moto,declare,земля,slave
7,электростимуляция,println,планшетный,set,counter,chromium,дед_мороз,грид,highload,таймаут,филамент,помещение,макро,telegram,models,мастер-класс,std,кожа,марс,uint32
8,хаб,admob,дикий_запад,define,начальство,firefox,hero,изоморфный,dusk,esxi,пират,гражданин,лазер,автоматизация,треугольник,тяньгун,siem,set,звезда,site
9,тепло,quick_charge,пленка,хост,clean,metrics,storage_spaces,crc,eps,background,abs,синхронизация,пожар,продавец,вебинар,видеоматериал,table,vector3,луна,amd_radeon


Также подберем параметры для ```tfidf_lda_model```.

In [30]:
for i in list(range(10, 100, 20)) + list(range(100, 400, 100)):
    print('n_topics: ', i)
    model = get_lda_model(corpus, id2word=new_dictionary2, num_topics=i, not_return_id2word=True)
    c, _ = get_coherence(model, new_dictionary2, new_ngrammed_texts, 20)
    print('---------')

n_topics:  10
Perplexity:  -28.55893904207621
Coherence:  0.36660456400876107
---------
n_topics:  30
Perplexity:  -68.34564091218033
Coherence:  0.4620563386218245
---------
n_topics:  50
Perplexity:  -95.87415357053017
Coherence:  0.4391479272885937
---------
n_topics:  70
Perplexity:  -117.98481797612159
Coherence:  0.45152784989808287
---------
n_topics:  90
Perplexity:  -142.1472063097758
Coherence:  0.42793024272375807
---------
n_topics:  100
Perplexity:  -155.85442506627237
Coherence:  0.4531327273243862
---------
n_topics:  200
Perplexity:  -255.46357832229944
Coherence:  0.4372549251552439
---------
n_topics:  300
Perplexity:  -345.6190266199971
Coherence:  0.39989136757693394
---------


In [34]:
for i in ['auto', 0.0001, 0.001, 0.01, 1]:
    print('eta: ', i)
    model = get_lda_model(corpus, id2word=new_dictionary2, num_topics=100, eta=i,
                                     not_return_id2word=True)
    c, _ = get_coherence(model, new_dictionary2, new_ngrammed_texts, 20)
    print('---------')

eta:  auto
Perplexity:  -156.55790518632816
Coherence:  0.4527422541920901
---------
eta:  0.0001
Perplexity:  -2255.16931943488
Coherence:  0.4382858419419209
---------
eta:  0.001
Perplexity:  -883.3172475456503
Coherence:  0.45566458980377106
---------
eta:  0.01
Perplexity:  -250.00579302149288
Coherence:  0.44667126432678456
---------
eta:  1
Perplexity:  -17.428973485674046
Coherence:  0.3248440789308701
---------


In [35]:
for i in range(5, 20, 5):
    print('passes: ', i)
    model = get_lda_model(corpus, id2word=new_dictionary2, num_topics=100, eta=0.001,
                                     passes=i, not_return_id2word=True)
    c, _ = get_coherence(model, new_dictionary2, new_ngrammed_texts, 20)
    print('---------')

passes:  5
Perplexity:  -873.6973041775655
Coherence:  0.4284101308418729
---------
passes:  10
Perplexity:  -372.9551387676833
Coherence:  0.46878352731528994
---------
passes:  15
Perplexity:  -255.1503151510508
Coherence:  0.47697853435410736
---------


In [41]:
tfidf_lda_model = get_lda_model(corpus, id2word=new_dictionary2,
                                                  num_topics=70, eta=0.0001,
                                                  passes=10, not_return_id2word=True)
c, topics = get_coherence(tfidf_lda_model, new_dictionary2, new_ngrammed_texts, 20)
topics

Perplexity:  -1077.8257232136307
Coherence:  0.45299134884316256


,Topic # 00,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,Topic # 11,Topic # 12,Topic # 13,Topic # 14,Topic # 15,Topic # 16,Topic # 17,Topic # 18,Topic # 19
0,react,fixed,болезнь_альцгеймер,сервер,dart,ssl,sharepoint,begin,tarantool,null,сертификат,выкуп,игра,цру,email,боль,public,порекомендовать,num,мозг
1,lt,ack,alias,язык,font,systemd,форум,hyper,bmc,varchar,o,nfc,сотрудник,приставка,рассылка,мышца,void,марс,лига,нейрон
2,this,ops,канва,сайт,hpe,хостер,clk,микросервис,python,select,антибиотик,фас,игрок,hdr,письмо,trello,git,лайфхак,universe,кора
3,компонент,long_long,yellow,файл,style,tdd,объектно-ориентированный,yield,c,where,хирург,mongodb,облачный,nes,подписчик,препарат,token,предпочитать,devops,art
4,props,книга,невесомость,элемент,appium,clion,дгу,float,cuda,nvarchar,sailfish,passport,безопасность,роман,conn,ibm_watson,org,книга,ротация,dusk
5,res,bim,подзапрос,сообщение,транзакция,proxy,int,лед,tox,from,yahoo,больной,бизнес,rom,role,врач,bot,scrum,беспилотник,gt
6,jsx,jenkins,неуловимый,интерфейс,symfony,nginx,smalltalk,резервный_копия,водород,image,ctfzone,хромосома,услуга,эмпатия,фольга,шея,nov,литература,rails,подсознание
7,return,do,ansible,библиотека,display,gen,assassin_creed,pla,riot,vector,бактерия,пульсометр,земля,удалять,mix,ооо,upload,час,oauth,summary
8,val,ветка,moikrug,алгоритм,document,ontap,криптовалютный,abs,лед,set,backup,вымогатель,закон,пост,elixir,val,fmt,зонд,кибератака,minutes
9,const,setstate,нло,класс,mvp,seagate,регулирование,spi,fixed,default,ctf,наказание,страна,бандеролька,пират,пациент,redmine,блокнот,коворкинг,xff


In [42]:
topics[['Topic # 03', 'Topic # 09', 'Topic # 12']]

,Topic # 03,Topic # 09,Topic # 12
0,сервер,null,игра
1,язык,varchar,сотрудник
2,сайт,select,игрок
3,файл,where,облачный
4,элемент,nvarchar,безопасность
5,сообщение,from,бизнес
6,интерфейс,image,услуга
7,библиотека,vector,земля
8,алгоритм,set,закон
9,класс,default,страна


Изначально:
* perplexity:  -90.86654043193215
* coherence:  0.5428529646433754

C tfidf:
* perplexity:  -1077.8257232136307
* coherence:  0.45299134884316256

То есть, значительно улучшилось значение ```perplexity```, но ухудшилось значение ```coherence```. Если смотреть на топики, то там появилось больше английских слов и, как кажется, темы стали менее осмысленными.

### Разложение матриц в sklearn

In [3]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [83]:
vectorizer = TfidfVectorizer(max_features=25000, ngram_range=(1, 2), min_df=5, max_df=0.3, lowercase=False, 
                             tokenizer=lambda x: x)
X = vectorizer.fit_transform(new_ngrammed_texts)

In [84]:
model = NMF(n_components=30)

In [85]:
model.fit(X)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=30, random_state=None, shuffle=False, solver='cd',
  tol=0.0001, verbose=0)

In [11]:
def get_nmf_topics(model, n_top_words):
    
    #id слов.
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(model.components_.shape[0]):
        
        #топ n слов для темы.
        words_ids = model.components_[i].argsort()[:-n_top_words - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);

In [87]:
get_nmf_topics(model, 10)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,...,Topic # 21,Topic # 22,Topic # 23,Topic # 24,Topic # 25,Topic # 26,Topic # 27,Topic # 28,Topic # 29,Topic # 30
0,бизнес,the,игра,мозг,lt,устройство,язык,public,космический,книга,...,робот,объект,windows,сеть,бот,товар,d,this,звук,end
1,сотрудник,and,игрок,ученый,lt lt,камера,программирование,void,спутник,час,...,ребенок,изображение,microsoft,связь,telegram,скидка,печать,function,сигнал,begin
2,клиент,for,игровой,пациент,class,смартфон,программист,public void,орбита,порекомендовать,...,робототехника,значение,linux,оператор,сообщение,цена,принтер,return,наушник,null
3,продукт,you,играть,клетка,lt class,аккумулятор,язык программирование,new,ракета,профессиональный,...,машина,алгоритм,azure,трафик,чат,магазин,материал,var,усилитель,from
4,деньги,with,персонаж,исследование,class lt,дисплей,php,string,марс,предпочитать,...,датчик,модель,обновление,интернет,bot,покупка,станок,this this,частота,select
5,рынок,that,steam,заболевание,name,usb,лекция,private,аппарат,путь,...,автомобиль,элемент,браузер,канал,api,покупатель,печатать,public function,музыка,then
6,заказчик,are,unity,врач,компонент,процессор,программа,static,земля,музыка,...,движение,текстура,visual_studio,dpi,канал,распродажа,производство,self,звуковой,запрос
7,crm,your,игра который,болезнь,lt name,экран,java,var,планета,слушать,...,дрон,метод,net,устройство,мессенджер,черный_пятница,изделие,const,акустический,varchar
8,опыт,not,движок,ген,элемент,корпус,перевод,класс,луна,пользоваться,...,обучение,точка,драйвер,станция,телегр,рубль,деталь,new,акустика,таблица
9,менеджер,system,жанр,организм,type,ноутбук,python,return,полет,внутренний проект,...,искусственный_интеллект,класс,ядро,абонент,сервис,акция,металл,компонент,искажение,where


In [13]:
def create_nmf(min_df=5, max_df=0.3, n_components=100):
    vectorizer = TfidfVectorizer(max_features=25000, ngram_range=(1, 2), min_df=min_df, max_df=max_df,
                                 lowercase=False, tokenizer=lambda x: x)
    X = vectorizer.fit_transform(new_ngrammed_texts)

    model = NMF(n_components=n_components)
    model.fit(X)

    print('reconstruction_err_: ', model.reconstruction_err_)
    
    return model, vectorizer

In [47]:
for i in range(20, 200, 50):
    print('n_components: ', i)
    model = create_nmf(min_df=5, max_df=0.3, n_components=i)

n_components:  20
reconstruction_err_:  60.912338404172885
n_components:  70
reconstruction_err_:  58.46140927864674
n_components:  120
reconstruction_err_:  56.7739008448871
n_components:  170
reconstruction_err_:  55.38307361526634


In [6]:
%%time
model1 = create_nmf(min_df=5, max_df=0.1, n_components=30)

reconstruction_err_:  61.07896438865475
CPU times: user 1min 19s, sys: 9.61 s, total: 1min 29s
Wall time: 1min 48s


In [7]:
%%time
model2 = create_nmf(min_df=10, max_df=0.3, n_components=30)

reconstruction_err_:  60.149303186322555
CPU times: user 2min 11s, sys: 7.76 s, total: 2min 18s
Wall time: 2min 12s


In [14]:
%%time
model, vectorizer = create_nmf(min_df=10, max_df=0.3, n_components=170)

reconstruction_err_:  55.114898403194545
CPU times: user 13min 30s, sys: 3min, total: 16min 30s
Wall time: 13min 22s


In [16]:
topics = get_nmf_topics(model, 10)
topics

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10,...,Topic # 90,Topic # 91,Topic # 92,Topic # 93,Topic # 94,Topic # 95,Topic # 96,Topic # 97,Topic # 98,Topic # 99
0,бизнес,class,игра,энергия,lt,файл,какой-то,public,марс,css,...,python,продукт,станок,linux,int,swift,сайт,сервис,двигатель,пациент
1,ита,lt class,игровой,солнечный,lt lt,папка,деньги,void,космический,веб,...,библиотека,дизайн,чпу,ядро,for int,let,посетитель,доступ,emdrive,врач
2,управление,class lt,играть,электричество,name,скрипт,жизнь,public void,аппарат,javascript,...,asyncio,дизайнер,производство,дистрибутив,for,анимация,интернет,микросервис,тяга,лечение
3,директор,класс,игра который,источник,lt name,строка,думать,private,луна,html,...,django,интерфейс,шпиндель,драйвер,float,value,контент,сервис который,испытание,болезнь
4,предприниматель,lt,steam,источник энергия,type,имя,вообще,override,полет,фронтенд,...,сообщество,design,изделие,ubuntu,int int,ios,ресурс,работа сервис,космический,вирус
5,руководитель,yii,движок,электроэнергия,name lt,директория,вещь,override public,миссия,стиль,...,pycharm,прототип,алюминий,патч,double,xcode,блокировка,сегмент,импульс,заболевание
6,kpi,background,unity,реактор,lt type,каталог,приходить,float,земля,web,...,c,экран,рабочий,поддержка,uint,objective,хостинг,онлайн,ток,препарат
7,развитие,gt,nintendo,мощность,value,file,заниматься,static,планета,font,...,курс,инструмент,деталь,debian,return,компиляция,сайт который,мкс,космос,вич
8,трансформация,font,игра это,китай,value lt,файл который,месяц,public static,космос,инструмент,...,инструмент,пользовательский,металл,ядро linux,long_long,func,https,интернет,вакуум,кровь
9,финансовый,display,приставка,панель,компонент,путь,никто,class,зонд,svg,...,программирование,иконка,изготовление,red_hat,char,компилятор,роскомнадзор,tarantool,стенд,watson


In [17]:
topics[['Topic # 04', 'Topic # 06', 'Topic # 09', 'Topic # 92', 'Topic # 96', 'Topic # 99']]

,Topic # 04,Topic # 06,Topic # 09,Topic # 92,Topic # 96,Topic # 99
0,энергия,файл,марс,станок,сайт,пациент
1,солнечный,папка,космический,чпу,посетитель,врач
2,электричество,скрипт,аппарат,производство,интернет,лечение
3,источник,строка,луна,шпиндель,контент,болезнь
4,источник энергия,имя,полет,изделие,ресурс,вирус
5,электроэнергия,директория,миссия,алюминий,блокировка,заболевание
6,реактор,каталог,земля,рабочий,хостинг,препарат
7,мощность,file,планета,деталь,сайт который,вич
8,китай,файл который,космос,металл,https,кровь
9,панель,путь,зонд,изготовление,роскомнадзор,watson


И в LDA, и в NMF есть хорошие осмысленные темы, но в LDA их намного меньше, чем в NMF.